# Updating Plasma and Convergence

As light travels through a real plasma, it has effects on the properties of the plasma due to light-matter
interactions as well as the presence of extra energy from the light. Additionally, as [previously discussed](../montecarlo/propagation.rst), properties of the plasma affect how light travels through it. This is a typical equilibrium problem. We solve for the plasma properties by finding a steady-state solution; that is, the actual plasma will be in a state such that the plasma state will not change as
light propagates through it, because the effects of the light on the plasma and the effects of the plasma on the
light are in equilibrium.

One of TARDIS's main goals is to determine this plasma state (as we need the actual plasma properties in order to
get an accurate spectrum). This is done in an iterative process. After each [Monte Carlo iteration](../montecarlo/index.rst) (which sends light through the supernova ejecta), TARDIS uses the [Monte Carlo estimators](../montecarlo/estimators.rst)
how the propagating light affects the plasma state, after which the plasma state is updated (as will be explained below and demonstrated in the code example). We do this many times, and attempt to have the plasma state converge
to the steady-state we are looking for. In fact, all but the last Monte Carlo iteration is used for this purpose
(after which TARDIS will have the necessary plasma state for its last iteration which calculates the spectrum).

<div class="alert alert-info">

Note

For all but the last iteration, TARDIS uses the number of Monte Carlo packets specified in the
[Monte Carlo configuration](../../io/configuration/components/montecarlo.rst) under the ``no_of_packets`` argument. This is because
a different number of packets may be necessary to calculate the spectrum as opposed to calculate the
plasma state.

</div>

After each iteration the values for radiative temperature and dilution factor are updated by calling the ``advance_state`` method on a ``Simulation`` object as will be shown below in the code example. The goal of this is to eventually have the radiative temperature and dilution factor converge to a single value so that the steady-state plasma state can be determined. To ensure that the simulation converges, TARDIS employs a convergence strategy. Currently, only one convergence strategy is available: damped convergence. This will be described in the following sections.

<div class="alert alert-info">

Note
    
Unless otherwise noted, all user-supplied quantities mentioned on this page are supplied in the [convergence section of the Monte Carlo configuration](../../io/configuration/components/montecarlo.rst#damped-convergence-strategy), which will be referenced as the convergence configuration.

</div>

## $T_\mathrm{rad}$ and $W$

The main way in which the plasma state is updated is by using the ``j`` and ``nu_bar`` estimators (see [here](../montecarlo/estimators.rst#j-and-bar-nu-estimators)) to update two of the key plasma inputs (see [Plasma](../setup/plasma/index.rst)): the radiative temperature $T_\mathrm{rad}$ and dilution factor $W$. Recall that each of these quantities takes on a different value in each shell.

Using the estimators $J$ and $\bar \nu$, we estimate these quantities using

$$T_{\mathrm{rad\ estimated}} = \frac{h}{k_{\mathrm{B}}} \frac{\pi^4}{360 \zeta(5)} \frac{\bar \nu}{J}$$

and

$$W_\mathrm{estimated} = \frac{\pi J}{\sigma_{\mathrm{R}} T_{\mathrm{rad\ estimated}}^4}$$

where $h$ is Planck's constant, $k_{\mathrm{B}}$ is Boltzmann's constant, $\sigma_{\mathrm{R}}$ is the Stefan–Boltzmann constant, and $\zeta$ is the Riemann zeta function. The equation for $W$ comes from the fact that the dilution factor is the ratio of the actual mean intensity to that of a blackbody, which is $J_{\mathrm{blackbody}}=\frac{\sigma_{\mathrm{R}} T^4}{\pi}$.

Recall (see [Implementation](../montecarlo/estimators.rst#implementation)), however, that when TARDIS stores these estimators, it leaves out the prefactor of $\frac{1}{4\pi V\Delta t}$ for computational ease. That is, $J=\frac{1}{4\pi V\Delta t}\sum_i \varepsilon_i l_i=\frac{1}{4\pi V\Delta t}*\mathrm{real\ j\ estimator}$, and $\bar \nu=\frac{1}{4\pi V\Delta t}\sum_i \varepsilon_i \nu_i l_i=\frac{1}{4\pi V\Delta t}*\mathrm{real\ nu\_ bar\ estimator}$. These factors are then included in our calculations; specifically, using the previous relations we have

$$T_\mathrm{rad\ estimated}= \frac{h}{k_{\mathrm{B}}} \frac{\pi^4}{360 \zeta(5)} \frac{\sum_i \varepsilon_i \nu_i l_i}{\sum_i \varepsilon_i l_i} = \frac{h}{k_{\mathrm{B}}} \frac{\pi^4}{360 \zeta(5)} \frac{\mathrm{real\ nu\_ bar\ estimator}}{\mathrm{real\ j\ estimator}}$$
and
$$W_\mathrm{estimated} = \frac{\sum_i \varepsilon_i l_i}{4\sigma_{\mathrm{R}} T_{\mathrm{rad\ estimated}}^4V\Delta t} = \frac{\mathrm{real\ j\ estimator}}{4\sigma_{\mathrm{R}} T_{\mathrm{rad\ estimated}}^4V\Delta t}.$$

While TARDIS can then update the plasma state using the estimated values, there is a good chance that these estimated values would “overshoot” the true value we want to converge to (for example, if the current value of the dilution factor in some cell the dilution factor is 0.4, and the true steady-state value TARDIS wants to find is 0.45, there is a good chance that the estimated value will be greater than 0.45). This could make the simulation take longer to converge or, at worst, make it so the simulation does not converge at all. To account for this, users can set (in the convergence configuration) a "damping constant" for both the radiative temperature ($d_{T_\mathrm{rad}}$) and the dilution factor ($d_W$). When ``advance_state`` is called, these quantities update as follows:

$$T_\mathrm{rad\ updated} = T_\mathrm{rad\ current} + d_{T_\mathrm{rad}}(T_\mathrm{rad\ estimated}-T_\mathrm{rad\ current})$$
    
and
    
$$ W_\mathrm{updated} = W_\mathrm{current} + d_W(W_\mathrm{estimated}-W_\mathrm{current}).$$

This means, for example, if the damping constant is 0.5, the updated value is halfway between the current value and the estimated value. If the damping constant is 0.7, the updated value is 70% of the way between the current value and the estimated value, and so on. **If the damping constant is 1, then the updated value is exactly the estimated value, and if the damping constant is zero, the value stays the same throughout the simulation and is not updated.**

The updated $T_\mathrm{rad}$ and $W$ are then used as inputs to the updated [plasma calculations](../setup/plasma/index.rst) which account for the effect of the Monte Carlo packets on the plasma state.

## $T_\mathrm{inner}$

The temperature of the inner boundary, $T_\mathrm{inner}$, plays a unique role in the simulation, as it is the primary determiner of the output luminosity. This is because the the luminosity of the inner boundary is proportional to $T_\mathrm{inner}^4$ (see [Energy Packet Initialization](../montecarlo/initialization.ipynb)). Thus, $T_\mathrm{inner}$ is updated throughout the simulation in order to match the output luminosity to the requested luminosity specified in the [supernova configuration](../../io/configuration/components/supernova.rst) between the bounds specified in the supernova configuration. However, there is not necessarily a quartic relationship between $T_\mathrm{inner}$ and the output luminosity, as changing $T_\mathrm{inner}$ also changes the frequency distribution of the initialized packets (once again see [Energy Packet Initialization](../montecarlo/initialization.ipynb)). This then affects the light-matter interactions, affecting which packets make it to the outer boundary, which also affects the output luminosity. Because of this, there is not an exact way to estimate $T_\mathrm{inner}$. To do this estimation, we use

$$T_\mathrm{inner\ estimated} = T_\mathrm{inner\ current} * \left(\frac{L_\mathrm{output}}{L_\mathrm{requested}}\right)^{\mathrm{t\_inner\_update\_exponent}}$$
    
where $L_\mathrm{output}$ is the output luminosity calculated by adding up the luminosity of each packet (see [Basic Spectrum Generation](../spectrum/basic.ipynb)) between the bounds specified in the [supernova configuration](../../io/configuration/components/supernova.rst), $L_\mathrm{requested}$ is the luminosity requested also in the supernova configuration (requested between those bounds previously mentioned), and ``t_inner_update_exponent`` is provided by the user in the convergence configuration. Note that what we are doing is "correcting" the previous value of the inner temperature by a factor of $\left(\frac{L_\mathrm{output}}{L_\mathrm{requested}}\right)^{\mathrm{t\_inner\_update\_exponent}}$. Note that if $\frac{L_\mathrm{output}}{L_\mathrm{requested}}$ is greater than 1, we want to lower $T_\mathrm{inner}$ as the output luminosity is too high, and vice versa if the ratio is less than 1. Thus ``t_inner_update_exponent`` should be negative. Naively one might set ``t_inner_update_exponent=-0.25``, however as a default TARDIS uses ``t_inner_update_exponent=-0.5`` as -0.25 may undershoot the correct $T_\mathrm{inner}$ because of its previously mentioned effects on the initial frequency distribution.

After calculating the estimated $T_\mathrm{inner}$, the quantity is updated using damped convergence with its own damping constant (once again set in the convergence configuration):

$$T_\mathrm{inner\ updated} = T_\mathrm{inner\ current} + d_{T_\mathrm{inner}}(T_\mathrm{inner\ estimated}-T_\mathrm{inner\ current}).$$

Once again, If the damping constant is 1, then the updated value is exactly the estimated value, and if the damping constant is zero, the value stays the same throughout the simulation and is not updated.

Additionally, because of the vast impact of $T_\mathrm{inner}$ on the simulation, one may want to update it less frequently -- i.e. allow $W$ and $T_\mathrm{rad}$ to reach a steady-state value for a particular $T_\mathrm{inner}$ before updating $T_\mathrm{inner}$. To do this, in the convergence configuration we set ``lock_t_inner_cycles``, which is the number of iterations to wait before updating $T_\mathrm{inner}$. It is set to 1 by default, meaning $T_\mathrm{inner}$ would be updated every iteration.

## Convergence Information

During the simulation, information about the how $T_\mathrm{rad}$, $W$, and $T_\mathrm{inner}$ are updated as well as a comparison of the total output luminosity and the requested luminosity are logged at the INFO level (see [Configuring the Logging Output for TARDIS](../../io/optional/logging_configuration.ipynb)) as shown in the code below, to give users a better idea of how the convergence process is working.

In addition, TARDIS allows for the displaying of convergence plots, which allows users to visualize the convergence process for $T_\mathrm{rad}$, $W$, $T_\mathrm{inner}$, and the total luminosity of the supernova being modeled. For more information, see [Convergence Plots](../../io/visualization/convergence_plot.ipynb).

## Convergence Criteria

TARDIS also allows users to stop the simulation if the simulation reaches a certain level of convergence, which is checked upon the call of the ``advance_state`` method. To enable this, users must set ``stop_if_converged=True`` in the convergence configuration. Also in the configuration, the quantities ``hold_iterations``, ``threshold``, and ``fraction`` are be specified to determine convergence as follows:

For the simulation to be considered to have converged, for ``hold_iterations`` successive iterations, the estimated values of $T_\mathrm{rad}$, $W$, and $T_\mathrm{inner}$ may differ from the previous value by a fraction of at most ``threshold`` in at least ``fraction`` fraction of the shells (for $T_\mathrm{inner}$, since there is only one value, the ``fraction`` part does not apply). For example, if ``hold_iterations=3``, ``threshold=0.05`` for all three quantities, and ``fraction=0.8``, the simulation will be considered to have converged if for 3 successive iterations the estimated values of $T_\mathrm{rad}$ and $W$ differ from the current respective values by at most 5% in at least 80% of the shells, *and* the estimated $T_\mathrm{inner}$ differs by at most 5%. See the [convergence configuration schema](../../io/configuration/components/montecarlo.rst#damped-convergence-strategy) for default values of these quantities.

<div class="alert alert-info">

Note
    
To determine convergence, we compare the estimated value, **not** the updated value (which is related to the estimated value via the damping constant), with the previous value. If $T_\mathrm{inner}$ is locked (see the previous section), the estimated value will still be calculated so convergence can be checked as usual.

</div>

<div class="alert alert-info">

Note
    
``hold_iterations`` and ``fraction`` are universal quantities, i.e. they are each a single value that applies to $T_\mathrm{rad}$ and $W$, and for ``hold_iterations`` also $T_\mathrm{inner}$. ``threshold``, on the other hand, is supplied for each quantity separately, so for instance you could require $T_\mathrm{rad}$ to differ by less than 1%, $W$ to differ by less than 3%, and $T_\mathrm{inner}$ to differ by less than 5% for convergence to be reached.

</div>


## Updating Other Quantities

Other quantities in the plasma state can depend on other estimators. Currently, this is only implemented for the ``j_blue`` estimator: If ``radiative_rates_type`` in the [plasma configuration](../../io/configuration/components/plasma.rst) is set to ``detailed``, the `j_blues` plasma property will will be replaced with the value of the $J^b_{lu}$ [estimator](../montecarlo/estimators.rst#j-b-lu-estimator) (the raw estimator times the factor of $\frac{ct_\mathrm{explosion}}{4\pi V \Delta t}$, once again see [Implementation](../montecarlo/estimators.rst#implementation)), which would then affect other plasma properties that depend on the `j_blues` values (see [Plasma](../setup/plasma/index.rst)). Otherwise, the `j_blues` values in the plasma are calculated as they typically are in the plasma calculations, and the $J^b_{lu}$ estimator is only used for the [formal integral](../spectrum/formal_integral.rst). Even in the former case, while the estimator does contribute to the updating of the plasma when the ``advance_state`` method is called, it does **not** contribute to the determination of if the simulation has converged, and it does **not** show up in the convergence logging or convergence plots.

## Code Example

We now show a detailed example of how the plasma is updated using the estimators after a Monte Carlo iteration. First, we import the necessary packages and set up a simulation (see [Setting Up the Simulation](../setup/index.rst)):

In [1]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.io.atom_data.util import download_atom_data
import numpy as np
from scipy.special import zeta

from astropy import units as u, constants as const

# We download the atomic data needed to run this notebook
download_atom_data('kurucz_cd23_chianti_H_He')

/usr/share/miniconda3/envs/tardis/lib/python3.8/site-packages/setuptools_scm/git.py:105: UserWarning: "/home/runner/work/tardis/tardis" is shallow and may cause errors
  warnings.warn(f'"{wd.path}" is shallow and may cause errors')


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
tardis_config = Configuration.from_yaml('tardis_example.yml')

# We manually put in the damping constants and t_inner_update_exponent for
# illustrative purposes:
d_t_rad = 0.5
d_w = 0.3
d_t_inner = 0.7
t_inner_update_exponent = -0.5

# We set the above values into the configuration:
tardis_config.montecarlo.convergence_strategy.t_rad.damping_constant = d_t_rad
tardis_config.montecarlo.convergence_strategy.w.damping_constant = d_w
tardis_config.montecarlo.convergence_strategy.t_inner.damping_constant = d_t_inner
tardis_config.montecarlo.convergence_strategy.t_inner_update_exponent = t_inner_update_exponent

In [3]:
sim = Simulation.from_config(tardis_config)

model = sim.model
plasma = sim.plasma
runner = sim.runner

/home/runner/work/tardis/tardis/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning: invalid value encountered in true_divide
  (g_lower * n_upper) / (g_upper * n_lower)


/home/runner/work/tardis/tardis/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning: invalid value encountered in true_divide
  (g_lower * n_upper) / (g_upper * n_lower)


We show the initial radiative temperature and dilution factor in each shell, the initial inner boundary temperature, and the initial electron densities in each shell:

In [4]:
model.t_rad

<Quantity [9926.50196546, 9911.63537753, 9896.81325339, 9882.03539385,
           9867.30160093, 9852.6116778 , 9837.96542884, 9823.36265956,
           9808.80317663, 9794.28678787, 9779.81330223, 9765.3825298 ,
           9750.99428177, 9736.64837045, 9722.34460927, 9708.08281272,
           9693.8627964 , 9679.68437699, 9665.54737223, 9651.45160094] K>

In [5]:
model.w

array([0.40039164, 0.33245223, 0.28966798, 0.2577141 , 0.23224568,
       0.21120466, 0.19341188, 0.17811402, 0.16479446, 0.1530809 ,
       0.14269498, 0.13342262, 0.12509541, 0.11757849, 0.11076215,
       0.10455605, 0.09888494, 0.09368554, 0.08890421, 0.08449514])

In [6]:
model.t_inner

<Quantity 9933.95199592 K>

In [7]:
plasma.electron_densities

0     2.865134e+09
1     2.182365e+09
2     1.678840e+09
3     1.303472e+09
4     1.020811e+09
5     8.059447e+08
6     6.411609e+08
7     5.137297e+08
8     4.144079e+08
9     3.364195e+08
10    2.747519e+08
11    2.256656e+08
12    1.863476e+08
13    1.546660e+08
14    1.289928e+08
15    1.080764e+08
16    9.094799e+07
17    7.685317e+07
18    6.520063e+07
19    5.552442e+07
dtype: float64

We set the number of packets and we run one iteration of the Monte Carlo simulation:

In [8]:
N_packets = 10000

# Using the commented out code below, we can also get the number of packets
# from the configuration -- try it out:
#N_packets = tardis_config.montecarlo.no_of_packets

sim.iterate(N_packets)

TqdmHBox(children=(HTML(value='Iterations:', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

TqdmHBox(children=(HTML(value='Packets:   ', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

We now show the values of the $J$ and $\bar \nu$ estimators, attaching their proper units (note that these are the raw estimators, and the factors of $\frac{1}{4\pi V \Delta t}$ etc are not included):

In [9]:
j_estimator = runner.j_estimator * (u.erg * u.cm) 
j_estimator

<Quantity [1.13264880e+14, 9.80619708e+13, 8.77127720e+13, 7.93178023e+13,
           7.33452674e+13, 6.69288783e+13, 6.42825236e+13, 6.09589567e+13,
           5.76244911e+13, 5.49744149e+13, 5.27742170e+13, 5.11516006e+13,
           4.94278177e+13, 4.78690453e+13, 4.66549201e+13, 4.55199706e+13,
           4.46151244e+13, 4.38293494e+13, 4.31417403e+13, 4.22137375e+13] cm erg>

In [10]:
nu_bar_estimator = runner.nu_bar_estimator * (u.erg * u.cm * u.Hz)
nu_bar_estimator

<Quantity [9.23669721e+28, 8.00440961e+28, 7.16701149e+28, 6.52233231e+28,
           6.04286574e+28, 5.48759864e+28, 5.29893354e+28, 5.02177670e+28,
           4.72808091e+28, 4.50654639e+28, 4.30918375e+28, 4.12731776e+28,
           3.97685768e+28, 3.81488727e+28, 3.68151763e+28, 3.55929590e+28,
           3.46700820e+28, 3.38472442e+28, 3.32955181e+28, 3.24241018e+28] cm erg Hz>

We show the values of $J$ and $\bar \nu$ including the prefactor:

In [11]:
V = model.volume
Delta_t = runner.calculate_time_of_simulation(model)
prefactor = 1 / (4 * np.pi * V * Delta_t)
J = prefactor * j_estimator
J

<Quantity [9.45518792e+10, 7.56726301e+10, 6.27559991e+10, 5.27611308e+10,
           4.54756787e+10, 3.87722846e+10, 3.48714873e+10, 3.10307383e+10,
           2.75797363e+10, 2.47840640e+10, 2.24500135e+10, 2.05659319e+10,
           1.88116707e+10, 1.72708598e+10, 1.59795038e+10, 1.48199674e+10,
           1.38245589e+10, 1.29412518e+10, 1.21519167e+10, 1.13555411e+10] erg / (cm2 s)>

In [12]:
nu_bar = prefactor * nu_bar_estimator
nu_bar

<Quantity [7.71066085e+25, 6.17685656e+25, 5.12779333e+25, 4.33856736e+25,
           3.74670964e+25, 3.17899750e+25, 2.87452457e+25, 2.55630094e+25,
           2.26291325e+25, 2.03168209e+25, 1.83311547e+25, 1.65942287e+25,
           1.51354724e+25, 1.37638808e+25, 1.26093507e+25, 1.15880235e+25,
           1.07429621e+25, 9.99389027e+24, 9.37848961e+24, 8.72211852e+24] erg Hz / (cm2 s)>

As mentioned [here](../montecarlo/estimators.rst#j-and-bar-nu-estimators), $\bar \nu$ is not truly the mean frequency (as you can see by units). We show the true mean frequency, which will, as expected, be in units of Hz.

In [13]:
nu_bar/J

<Quantity [8.15495252e+14, 8.16260325e+14, 8.17100102e+14, 8.22303711e+14,
           8.23893068e+14, 8.19914928e+14, 8.24319463e+14, 8.23796366e+14,
           8.20498509e+14, 8.19753407e+14, 8.16532010e+14, 8.06879493e+14,
           8.04578852e+14, 7.96942416e+14, 7.89095260e+14, 7.81919640e+14,
           7.77092577e+14, 7.72250664e+14, 7.71770398e+14, 7.68093605e+14] Hz>

We show the calculations of the estimated and updated $T_\mathrm{rad}$ and $W$. Note that the ``decompose()`` method is used to simplify the units:

In [14]:
t_rad_estimated = ( (const.h.cgs / const.k_B.cgs) 
                   * (np.pi**4 / (360 * zeta(5, 1)))
                   * (nu_bar_estimator / j_estimator) ).decompose()
t_rad_estimated

<Quantity [10212.75275146, 10222.33404753, 10232.85089092, 10298.01763651,
           10317.92175646, 10268.10201688, 10323.26166378, 10316.71071495,
           10275.4104165 , 10266.07922661, 10225.73647129, 10104.85437906,
           10076.04258982,  9980.40864261,  9882.13576533,  9792.27278261,
            9731.8216644 ,  9671.18456832,  9665.17002029,  9619.1241619 ] K>

In [15]:
t_rad_updated = model.t_rad + d_t_rad * (t_rad_estimated - model.t_rad)
t_rad_updated

<Quantity [10069.62735846, 10066.98471253, 10064.83207215, 10090.02651518,
           10092.61167869, 10060.35684734, 10080.61354631, 10070.03668726,
           10042.10679657, 10030.18300724, 10002.77488676,  9935.11845443,
            9913.51843579,  9858.52850653,  9802.2401873 ,  9750.17779766,
            9712.8422304 ,  9675.43447265,  9665.35869626,  9635.28788142] K>

In [16]:
w_estimated = ( j_estimator / (4 * const.sigma_sb.cgs * t_rad_estimated**4 * V * Delta_t) ).decompose()
w_estimated

<Quantity [0.4815458 , 0.38395229, 0.31710812, 0.25991915, 0.22230487,
           0.19324105, 0.17011454, 0.15176296, 0.13706675, 0.12362115,
           0.11375668, 0.10928659, 0.10111279, 0.09644048, 0.09283222,
           0.0893001 , 0.08539126, 0.08195893, 0.0771517 , 0.07348597]>

In [17]:
w_updated = model.w + d_w * (w_estimated - model.w)
w_updated

<Quantity [0.42473789, 0.34790225, 0.29790002, 0.25837561, 0.22926344,
           0.20581558, 0.18642268, 0.1702087 , 0.15647615, 0.14424298,
           0.13401349, 0.12618181, 0.11790062, 0.11123709, 0.10538317,
           0.09997927, 0.09483683, 0.09016756, 0.08537846, 0.08119239]>

We show the output and requested luminosities, and use these to calculate the estimated and updated $T_\mathrm{inner}$:

In [18]:
# The output luminosity is calculated between the bounds specified below
nu_lower = 0
nu_upper = np.inf

# Using the commented out code below, we can also get the frequency bounds
# from the configuration -- try it out (note we convert between wavelength
# and frequency, and thus the order switches):
#nu_lower = tardis_config.supernova.luminosity_wavelength_end.to(u.Hz, u.spectral)
#nu_upper = tardis_config.supernova.luminosity_wavelength_start.to(u.Hz, u.spectral)

L_output = runner.calculate_emitted_luminosity(0,np.inf)
L_output

<Quantity 7.98068173e+42 erg / s>

In [19]:
L_requested = sim.luminosity_requested
L_requested

<Quantity 1.05927636e+43 erg / s>

In [20]:
t_inner_estimated = model.t_inner * (L_output / L_requested)**t_inner_update_exponent
t_inner_estimated

<Quantity 11444.76030204 K>

In [21]:
t_inner_updated = model.t_inner + d_t_inner * (t_inner_estimated - model.t_inner)
t_inner_updated

<Quantity 10991.51781021 K>

We now advance the state of the simulation based on the estimators. This will also display a summary of the updated values of $T_\mathrm{rad}$ and $W$. Additionally, the ``advance_state()`` method checks for convergence and returns the convergence status as shown below.

In [22]:
sim.advance_state()

Shell No.,t_rad,next_t_rad,w,next_w
0,9.93e+03,1.01e+04,0.4,0.425
5,9.85e+03,1.01e+04,0.211,0.206
10,9.78e+03,1e+04,0.143,0.134
15,9.71e+03,9.75e+03,0.105,0.1


/home/runner/work/tardis/tardis/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning: invalid value encountered in true_divide
  (g_lower * n_upper) / (g_upper * n_lower)


False

Finally, we show the full updated $T_\mathrm{rad}$, $W$, and $T_\mathrm{inner}$, as well as the updated electron densities which were updated along with the rest of the plasma based on the new values for $T_\mathrm{rad}$, $W$, and $T_\mathrm{inner}$. Compare these with our calculations above and with the initial values at the beginning of the code example!

In [23]:
model.t_rad

<Quantity [10069.62596067, 10066.98331343, 10064.83067161, 10090.02510572,
           10092.6102665 , 10060.35544197, 10080.61213339, 10070.03527523,
           10042.1053902 , 10030.18160214, 10002.77348719,  9935.1170714 ,
            9913.51705671,  9858.52714054,  9802.23883475,  9750.17645742,
            9712.84089843,  9675.43314898,  9665.35737341,  9635.28656487] K>

In [24]:
model.w

array([0.42473789, 0.34790225, 0.29790003, 0.25837562, 0.22926344,
       0.20581558, 0.18642268, 0.1702087 , 0.15647615, 0.14424298,
       0.1340135 , 0.12618181, 0.11790063, 0.11123709, 0.10538317,
       0.09997927, 0.09483683, 0.09016756, 0.08537846, 0.08119239])

In [25]:
model.t_inner

<Quantity 10991.51781021 K>

In [26]:
plasma.electron_densities

0     2.879194e+09
1     2.192349e+09
2     1.686013e+09
3     1.309369e+09
4     1.025252e+09
5     8.089293e+08
6     6.437049e+08
7     5.156985e+08
8     4.158632e+08
9     3.375895e+08
10    2.756466e+08
11    2.262224e+08
12    1.867888e+08
13    1.549413e+08
14    1.291439e+08
15    1.081434e+08
16    9.097366e+07
17    7.684827e+07
18    6.520044e+07
19    5.551064e+07
dtype: float64